<a href="https://colab.research.google.com/github/s1c5000/kaggle/blob/main/pima-indians-diabetes/8_RandomSearchCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터 로드와 split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sklearn/diabetes_feature.csv')
df.shape

(768, 16)

In [ ]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome', 'Pregnancies_high',
       'Age_low', 'Age_middle', 'Age_high', 'Insulin_nan', 'Insulin_log',
       'low_glu_insulin'],
      dtype='object')

In [ ]:
x = df[['Glucose', 'BloodPressure', 'SkinThickness',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Pregnancies_high',
       'Insulin_nan','low_glu_insulin']]
x.shape

(768, 9)

In [ ]:
y = df['Outcome']
y.shape

(768,)

sklearn에서 제공하는 model_selection의 train_test_split으로 테스트와 학습 나누어 보겠다

In [ ]:
# random 하게 세트들을 가져온다
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

In [ ]:
X_train

,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,Pregnancies_high,Insulin_nan,low_glu_insulin
60,84,0,0,0.0,0.304,21,False,102.5,True
618,112,82,24,28.2,1.282,50,True,169.5,False
346,139,46,19,28.7,0.654,22,False,83.0,False
294,161,50,0,21.9,0.254,65,False,102.5,False
231,134,80,37,46.2,0.238,46,False,370.0,False
...,...,...,...,...,...,...,...,...,...
71,139,64,35,28.6,0.411,26,False,140.0,False
106,96,122,0,22.4,0.207,27,False,102.5,True
270,101,86,37,45.6,1.136,38,True,169.5,False
435,141,0,0,42.4,0.205,29,False,169.5,False


In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((614, 9), (614,), (154, 9), (154,))

## 여러개의 알고리즘을 사용해서 비교하기

In [ ]:
# 여러개의 알고리즘을 선언한 뒤 하나의 변수에 리스트 형태로 넣어준다

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

estimators = [DecisionTreeClassifier(random_state=42),
              RandomForestClassifier(random_state=42),
              GradientBoostingClassifier(random_state=42)
]

estimators

[DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                        max_depth=None, max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort='deprecated',
                        random_state=42, splitter='best'),
 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='gini', max_depth=None, max_features='auto',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=100,
                        n_jobs=None, oob_score=False, random_state=42, verbose=0,
                        warm_start=False),
 GradientBoost

In [ ]:
# for문으로 알고리즘 이름 출력가능
for estimator in estimators:
  print(estimator.__class__.__name__)
RandomizedSearchCV( )

DecisionTreeClassifier
RandomForestClassifier
GradientBoostingClassifier


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

max_depth = np.random.randint(2,20,10)
max_features = np.random.uniform(0.3, 1.0, 10)
# parameter 값들을 이름으로 넣는다
param_distributions = {'max_depth' : max_depth, 
                      'max_features' : max_features}
                      
results = []
# 알고리즘이 순회하면서 돈다
for estimator in estimators:
  # RF와 GB는 n_estimators parameter가 있으니 넣어준다
  if estimator.__class__.__name__ != 'DecisionTreeClassifier':
    param_distributions['n_estimators'] = np.random.randint(100,200,10)

  clf = RandomizedSearchCV(estimator, 
                          param_distributions,
                          n_iter=100, 
                          scoring='accuracy',
                          n_jobs= -1, 
                          cv=5, 
                          verbose=2
                          )
  
  clf.fit(X_train, y_train)
  results.append([estimator.__class__.__name__, clf.best_params_, clf.best_score_,clf.score(X_test,y_test), clf.cv_results_])


Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 396 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    2.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:   41.7s
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  2.2min finished


Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:   40.7s
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  2.0min finished


In [ ]:
results_df = pd.DataFrame(results, columns=['estimator', 'best_params', 'train_score', 'test_score','cv_result'])
results_df
# cv 와 n_iter 값들을 올리면 더 좋은 모델을 얻을 수 있다.
# 결과를 보고 parameter값들을 수정해간다.

,estimator,best_params,train_score,test_score,cv_result
0,DecisionTreeClassifier,"{'max_features': 0.6455929642522225, 'max_dept...",0.866480,0.837662,"{'mean_fit_time': [0.005972146987915039, 0.006..."
1,RandomForestClassifier,"{'n_estimators': 187, 'max_features': 0.645592...",0.903985,0.863636,"{'mean_fit_time': [0.5050442218780518, 0.54792..."
2,GradientBoostingClassifier,"{'n_estimators': 171, 'max_features': 0.448809...",0.907224,0.863636,"{'mean_fit_time': [0.42422094345092776, 0.4292..."


In [ ]:
pd.DataFrame(results_df.loc[1, "cv_result"]).sort_values(by="rank_test_score").head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_max_features,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
29,0.618882,0.018619,0.032172,0.001486,187,0.645593,12,"{'n_estimators': 187, 'max_features': 0.645592...",0.878049,0.934959,0.853659,0.902439,0.950820,0.903985,0.035667,1
20,0.471137,0.022420,0.023326,0.002940,123,0.854532,7,"{'n_estimators': 123, 'max_features': 0.854531...",0.886179,0.934959,0.869919,0.886179,0.942623,0.903972,0.029144,2
4,0.521181,0.011900,0.028729,0.002801,168,0.671337,6,"{'n_estimators': 168, 'max_features': 0.671336...",0.869919,0.926829,0.878049,0.902439,0.942623,0.903972,0.027750,3
91,0.567271,0.012113,0.029811,0.000531,187,0.645593,19,"{'n_estimators': 187, 'max_features': 0.645592...",0.878049,0.934959,0.861789,0.902439,0.942623,0.903972,0.031330,3
15,0.541652,0.028870,0.030431,0.003390,168,0.671337,6,"{'n_estimators': 168, 'max_features': 0.671336...",0.869919,0.926829,0.878049,0.902439,0.942623,0.903972,0.027750,3
